In [1]:
import numpy as np 
import pandas as pd

In [2]:
dF = pd.read_csv('data//csvs//dataframeV1.csv', index_col=0)
dF = dF.drop(['id', 'uri'], axis = 1)
dF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,label
0,0.2750,0.1570,7,-18.752,1,0.0636,0.890,0.842,0.186,0.3040,73.289,152280,4,classic
1,0.2210,0.1260,0,-25.427,1,0.0447,0.989,0.897,0.102,0.2160,133.630,139307,4,classic
2,0.2890,0.0306,9,-30.790,0,0.0446,0.987,0.911,0.102,0.1180,125.610,212067,3,classic
3,0.0753,0.0700,2,-27.272,1,0.0440,0.918,0.947,0.146,0.0625,79.801,365147,4,classic
4,0.1300,0.1580,2,-16.132,1,0.0350,0.748,0.924,0.100,0.0998,85.031,302093,4,classic


In [3]:
dF.label = pd.Categorical(dF.label)
dF['Y'] = dF.label.cat.codes
dF = dF.drop(['label'], axis = 1)
dF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Y
0,0.2750,0.1570,7,-18.752,1,0.0636,0.890,0.842,0.186,0.3040,73.289,152280,4,0
1,0.2210,0.1260,0,-25.427,1,0.0447,0.989,0.897,0.102,0.2160,133.630,139307,4,0
2,0.2890,0.0306,9,-30.790,0,0.0446,0.987,0.911,0.102,0.1180,125.610,212067,3,0
3,0.0753,0.0700,2,-27.272,1,0.0440,0.918,0.947,0.146,0.0625,79.801,365147,4,0
4,0.1300,0.1580,2,-16.132,1,0.0350,0.748,0.924,0.100,0.0998,85.031,302093,4,0


In [4]:
from DecisionTree.tree import Tree


In [5]:
t = Tree()